In [1]:
import os
import json
import subprocess
import argparse
import numpy as np
import pandas as pd
from skimage.measure import find_contours


class CocoDatasetHandler:
    def __init__(self, jsonpath, imgpath):
        with open(jsonpath, 'r') as jsonfile:
            ann = json.load(jsonfile)

        images = pd.DataFrame.from_dict(ann['images']).set_index('id')
        annotations = pd.DataFrame.from_dict(
            ann['annotations']).set_index('id')
        categories = pd.DataFrame.from_dict(ann['categories']).set_index('id')

        annotations = annotations.merge(
            images, left_on='image_id', right_index=True)
        annotations = annotations.merge(
            categories, left_on='category_id', right_index=True)
        annotations = annotations.assign(
            shapes=annotations.apply(self.coco2shape, axis=1))
        self.annotations = annotations
        self.labelme = {}

        self.imgpath = imgpath
        self.images = pd.DataFrame.from_dict(
            ann['images']).set_index('file_name')

    def coco2shape(self, row):
        if row.iscrowd == 1:
            shapes = self.rle2shape(row)
        elif row.iscrowd == 0:
            shapes = self.polygon2shape(row)
        return shapes

    def rle2shape(self, row):
        rle, shape = row['segmentation']['counts'], row['segmentation']['size']
        mask = self._rle_decode(rle, shape)
        padded_mask = np.zeros(
            (mask.shape[0]+2, mask.shape[1]+2),
            dtype=np.uint8,
        )
        padded_mask[1:-1, 1:-1] = mask
        points = find_contours(mask, 0.5)
        shapes = [
            [[int(point[1]), int(point[0])] for point in polygon]
            for polygon in points
        ]
        return shapes

    def _rle_decode(self, rle, shape):
        mask = np.zeros([shape[0] * shape[1]], np.bool)
        for idx, r in enumerate(rle):
            if idx < 1:
                s = 0
            else:
                s = sum(rle[:idx])
            e = s + r
            if e == s:
                continue
            assert 0 <= s < mask.shape[0]
            assert 1 <= e <= mask.shape[0], "shape: {}  s {}  e {} r {}".format(
                shape, s, e, r)
            if idx % 2 == 1:
                mask[s:e] = 1
        # Reshape and transpose
        mask = mask.reshape([shape[1], shape[0]]).T
        return mask

    def polygon2shape(self, row):
        # shapes: (n_polygons, n_points, 2)
        shapes = [
            [[float(points[2*i]), float(points[2*i+1])]
             for i in range(len(points)//2)]
            for points in row.segmentation
        ]
        return shapes

    def coco2labelme(self):
        fillColor = [255, 0, 0, 128]
        lineColor = [0, 255, 0, 128]

        groups = self.annotations.groupby('file_name')
        for file_idx, (filename, df) in enumerate(groups):
            record = {
                'imageData': None,
                'fillColor': fillColor,
                'lineColor': lineColor,
                'imagePath': filename,
                'imageHeight': int(self.images.loc[filename].height),
                'imageWidth': int(self.images.loc[filename].width),
            }
            record['shapes'] = []

            instance = {
                'line_color': None,
                'fill_color': None,
                'shape_type': "polygon",
            }
            for inst_idx, (_, row) in enumerate(df.iterrows()):
                for polygon in row.shapes:
                    copy_instance = instance.copy()
                    copy_instance.update({
                        'label': row['name'],
                        'group_id': inst_idx,
                        'points': polygon
                    })
                    record['shapes'].append(copy_instance)
            if filename not in self.labelme.keys():
                self.labelme[filename] = record

    def save_labelme(self, file_names, dirpath, save_json_only=False):
        if not os.path.exists(dirpath):
            os.makedirs(dirpath)
#         else:
#             raise ValueError(f"{dirpath} has existed")
        print(file_names)
        for file in file_names:
            filename = os.path.basename(os.path.splitext(file)[0])
            
            
            with open(os.path.join(dirpath, filename+'.json'), 'w') as jsonfile:
                json.dump(self.labelme[file], jsonfile,
                          ensure_ascii=True, indent=2)
#             if not save_json_only:
#                 subprocess.call(
#                     ['cp', os.path.join(self.imgpath, file), dirpath])


if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        prog='Coco2Labelme Converter',
        description='Convert Coco annotations to labelme format',
        epilog='Code adapted from travishsu gist'
    )

#     parser.add_argument('coco_json', help='D:/test 10/train.json')
#     parser.add_argument('data_path', help='D:/test 10/New folder/')
#     parser.add_argument('output_dir', help='D:/test65/results/')

#     args = parser.parse_args()

    ds = CocoDatasetHandler('D:/test 10/train.json','D:/opaque/')
    ds.coco2labelme()
    ds.save_labelme(ds.labelme.keys(), 'D:/test65/results/')

dict_keys(['70_color_image1.bmp', '70_color_image112.bmp', '70_color_image117.bmp', '70_color_image118.bmp', '70_color_image122.bmp', '70_color_image127.bmp', '70_color_image56.bmp', '70_color_image57.bmp', '70_color_image62.bmp', '70_color_image70.bmp', '70_color_image71.bmp', '70_color_image72.bmp', '70_color_image74.bmp', '70_color_image75.bmp', '70_color_image76.bmp', '70_color_image77.bmp', '70_color_image78.bmp', '70_color_image79.bmp', '70_color_image80.bmp', '70_color_image81.bmp', '70_color_image83.bmp', '70_color_image84.bmp', '70_color_image85.bmp', '70_color_image86.bmp', '70_color_image87.bmp', '70_color_image88.bmp', '70_color_image89.bmp', '70_color_image90.bmp', '70_image_4000.bmp', '70_image_4001.bmp', '70_image_4002.bmp', '70_image_4003.bmp', '70_image_4004.bmp', '70_image_4005.bmp', '70_image_4006.bmp', '70_image_4007.bmp', '70_image_4008.bmp', '70_image_4009.bmp', '70_image_4010.bmp', '70_image_4011.bmp', '70_image_4012.bmp', '70_image_4013.bmp', '70_image_4014.bmp'

In [ ]:
import os
import json
import subprocess
import argparse
import numpy as np
import pandas as pd
from skimage.measure import find_contours


class CocoDatasetHandler:
    def __init__(self, jsonpath, imgpath):
        with open(jsonpath, 'r') as jsonfile:
            ann = json.load(jsonfile)

        images = pd.DataFrame.from_dict(ann['images']).set_index('id')
        annotations = pd.DataFrame.from_dict(
            ann['annotations']).set_index('id')
        categories = pd.DataFrame.from_dict(ann['categories']).set_index('id')

        annotations = annotations.merge(
            images, left_on='image_id', right_index=True)
        annotations = annotations.merge(
            categories, left_on='category_id', right_index=True)
        annotations = annotations.assign(
            shapes=annotations.apply(self.coco2shape, axis=1))
        self.annotations = annotations
        self.labelme = {}

        self.imgpath = imgpath
        self.images = pd.DataFrame.from_dict(
            ann['images']).set_index('file_name')

    def coco2shape(self, row):
        if row.iscrowd == 1:
            shapes = self.rle2shape(row)
        elif row.iscrowd == 0:
            shapes = self.polygon2shape(row)
        return shapes

    def rle2shape(self, row):
        rle, shape = row['segmentation']['counts'], row['segmentation']['size']
        mask = self._rle_decode(rle, shape)
        padded_mask = np.zeros(
            (mask.shape[0]+2, mask.shape[1]+2),
            dtype=np.uint8,
        )
        padded_mask[1:-1, 1:-1] = mask
        points = find_contours(mask, 0.5)
        shapes = [
            [[int(point[1]), int(point[0])] for point in polygon]
            for polygon in points
        ]
        return shapes

    def _rle_decode(self, rle, shape):
        mask = np.zeros([shape[0] * shape[1]], np.bool)
        for idx, r in enumerate(rle):
            if idx < 1:
                s = 0
            else:
                s = sum(rle[:idx])
            e = s + r
            if e == s:
                continue
            assert 0 <= s < mask.shape[0]
            assert 1 <= e <= mask.shape[0], "shape: {}  s {}  e {} r {}".format(
                shape, s, e, r)
            if idx % 2 == 1:
                mask[s:e] = 1
        # Reshape and transpose
        mask = mask.reshape([shape[1], shape[0]]).T
        return mask

    def polygon2shape(self, row):
        # shapes: (n_polygons, n_points, 2)
        shapes = [
            [[float(points[2*i]), float(points[2*i+1])]
             for i in range(len(points)//2)]
            for points in row.segmentation
        ]
        return shapes

    def coco2labelme(self):
        fillColor = [255, 0, 0, 128]
        lineColor = [0, 255, 0, 128]

        groups = self.annotations.groupby('file_name')
        for file_idx, (filename, df) in enumerate(groups):
            record = {
                'imageData': None,
                'fillColor': fillColor,
                'lineColor': lineColor,
                'imagePath': filename,
                'imageHeight': int(self.images.loc[filename].height),
                'imageWidth': int(self.images.loc[filename].width),
            }
            record['shapes'] = []

            instance = {
                'line_color': None,
                'fill_color': None,
                'shape_type': "polygon",
            }
            for inst_idx, (_, row) in enumerate(df.iterrows()):
                for polygon in row.shapes:
                    copy_instance = instance.copy()
                    copy_instance.update({
                        'label': row['name'],
                        'group_id': inst_idx,
                        'points': polygon
                    })
                    record['shapes'].append(copy_instance)
            if filename not in self.labelme.keys():
                self.labelme[filename] = record

    def save_labelme(self, file_names, dirpath, save_json_only=False):
        if not os.path.exists(dirpath):
            os.makedirs(dirpath)
#         else:
#             raise ValueError(f"{dirpath} has existed")
        print(file_names)
        for file in file_names:
            filename = os.path.basename(os.path.splitext(file)[0])
            
            
            with open(os.path.join(dirpath, filename+'.json'), 'w') as jsonfile:
                json.dump(self.labelme[file], jsonfile,
                          ensure_ascii=True, indent=2)
#             if not save_json_only:
#                 subprocess.call(
#                     ['cp', os.path.join(self.imgpath, file), dirpath])


if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        prog='Coco2Labelme Converter',
        description='Convert Coco annotations to labelme format',
        epilog='Code adapted from travishsu gist'
    )

#     parser.add_argument('coco_json', help='D:/test 10/train.json')
#     parser.add_argument('data_path', help='D:/test 10/New folder/')
#     parser.add_argument('output_dir', help='D:/test65/results/')

#     args = parser.parse_args()

    ds = CocoDatasetHandler('D:/test 10/train.json','D:/opaque/')
    ds.coco2labelme()
    ds.save_labelme(ds.labelme.keys(), 'D:/test65/results/')